Beyond Accuracy: Precision and Recall

https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from IPython.core.pylabtools import figsize
figsize(10, 8)

## Confusnion Matrix Numbers

In [0]:
results = pd.DataFrame({'fn': [0,   2,  4,  8, 11, 13, 18,  21,  33, 40, 50],
                        'fp': [50, 47, 40, 31, 23, 16, 12,  11,   4,  3,  0],
                        'threshold': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                        'tn': [0,   3,  9, 16, 22, 29, 34,  38,  43, 45, 50],
                        'tp': [50, 48, 47, 45, 44, 42, 36,  30,  20, 12, 0],
                       })

In [0]:
results

In [0]:
i = results.iterrows()
val = next(i)[1]
print(val)
fn, fp, t, tn, tp = val
print(fn)

## Calculate Precision, Recall, F1, TPR, FPR

In [0]:
def calculate_metrics(results):
  roc = pd.DataFrame(index=results['threshold'], columns=['recall', 'precision', 'f1', 'tpr', 'fpr'])

  for i in results.iterrows():
    fn, fp, t, tn, tp = i[1]
    assert tp + fp + tn + fn == 100, 'Patients must add up to 100'

    recall = tp / (tp + fn)

    if tp == fp == 0:
      precision = 0
      true_positive_rate = 0
    else:
      precision = tp / (tp + fp)
      true_positive_rate = tp / (tp + fn)
    
    if precision == recall == 0:
      f1 = 0
    else:
      f1 = 2 * (precision * recall) / (precision + recall)

    false_positive_rate = fp / (fp + tn)

    roc.loc[t, 'recall'] = recall
    roc.loc[t, 'precision'] = precision
    roc.loc[t, 'f1'] = f1
    roc.loc[t, 'tpr'] = true_positive_rate
    roc.loc[t, 'fpr'] = false_positive_rate
  
  return roc

In [0]:
roc = calculate_metrics(results)
roc.reset_index()

## Receiver Operating Characteristic Curve

In [0]:
plt.style.use('seaborn-dark-palette')
thresholds = [str(t) for t in results['threshold']]
plt.plot(roc['fpr'], roc['tpr'], 'bo-', label = 'model')
plt.plot(list(np.linspace(0, 1, num = 10)), list(np.linspace(0, 1, num = 10)), 'ro--', label = 'naive classifier')
for x, y, s in zip(roc['fpr'], roc['tpr'], thresholds):
    plt.text(x - 0.04,y + 0.02, s, fontdict={'size': 14})

plt.legend(prop={'size':14})
plt.ylabel('True Positive Rate', size = 16)
plt.xlabel('False Positive Rate', size = 16)
plt.title('Receiver Operating Characteristic Curve', size = 20);